# Lab 3 In-Class: VIX vs Realised Volatility

Exploring the Volatility Risk Premium with Bloomberg

> **In-Class Lab — Bloomberg Terminal Required**
>
> This lab is designed for the **Financial Innovation Lab** with
> Bloomberg Terminal access.
>
> **Duration**: 2 hours  
> **Prerequisites**: Complete Lab 3 Homework (GARCH estimation)

## Learning Objectives

By the end of this lab, you should be able to:

1.  **Extract** VIX and equity price data from Bloomberg
2.  **Calculate** realised volatility using different methods
3.  **Quantify** the volatility risk premium
4.  **Analyse** how the relationship varies across market regimes
5.  **Connect** implied and realised volatility to trading strategies

## Background: VIX and the Volatility Risk Premium

### What is the VIX?

The **VIX** (CBOE Volatility Index) measures the market’s expectation of
30-day volatility, derived from S&P 500 index option prices. Key
features:

-   **Forward-looking**: Based on option prices, not historical returns
-   **Risk-neutral**: Incorporates risk preferences of market
    participants
-   **Annualised**: Expressed as annual percentage (e.g., VIX = 20 means
    expected 20% annualised volatility)

### The Volatility Risk Premium

**Realised volatility** is what actually happens over the next 30 days.

**Volatility Risk Premium** = VIX - Realised Volatility

Historically, VIX tends to be **higher** than subsequent realised
volatility. This “premium” compensates sellers of volatility for taking
on risk.

> **Why Does the Premium Exist?**
>
> -   **Insurance demand**: Investors buy options for downside
>     protection
> -   **Crash risk**: The premium compensates for rare but severe events
> -   **Leverage constraints**: Not everyone can arbitrage the
>     difference

## Part 1: Bloomberg Data Extraction

### Step 1.1: Access Bloomberg Terminal

1.  Log in to the Bloomberg Terminal
2.  Open the **Excel Add-in** (Bloomberg ribbon in Excel)

### Step 1.2: Extract VIX History

Use the **BDH** (Bloomberg Data History) function to extract VIX data.

**In Excel:**

    Cell A1: =BDH("VIX Index", "PX_LAST", "20190101", "20241231", "Days", "A")

This retrieves: - **Ticker**: VIX Index - **Field**: Last price
(PX_LAST) - **Period**: 1 Jan 2019 to 31 Dec 2024 - **Frequency**: Daily
(“A” for actual trading days)

### Step 1.3: Extract S&P 500 Prices

Extract SPX (S&P 500 Index) prices to calculate realised volatility:

    Cell C1: =BDH("SPX Index", "PX_LAST", "20190101", "20241231", "Days", "A")

### Step 1.4: Save and Import to Python

1.  Copy the data to a clean worksheet
2.  Save as **CSV** (e.g., `vix_spx_data.csv`)
3.  Upload to your Python environment

## Part 2: Python Analysis

### Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Load the Bloomberg data
# Replace with your actual file path
df = pd.read_csv('vix_spx_data.csv', parse_dates=['Date'], index_col='Date')

# Ensure columns are named correctly
# df.columns = ['VIX', 'SPX']

print(f"Data period: {df.index[0]} to {df.index[-1]}")
print(f"Observations: {len(df)}")

### Task 2.1: Calculate Realised Volatility

**Realised volatility** is calculated from actual returns. The standard
approach:

$$\text{RV}_t = \sqrt{\frac{252}{n} \sum_{i=1}^{n} r_{t-i}^2}$$

where $n$ = 21 trading days (approximately 1 month).

In [ ]:
# Calculate daily returns
df['Return'] = df['SPX'].pct_change()

# 21-day realised volatility (annualised)
df['RV_21'] = df['Return'].rolling(21).std() * np.sqrt(252) * 100

# Also calculate 30-day for comparison
df['RV_30'] = df['Return'].rolling(30).std() * np.sqrt(252) * 100

print(df[['VIX', 'RV_21', 'RV_30']].describe())

### Task 2.2: Visualise VIX vs Realised Volatility

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1: Time series
axes[0].plot(df.index, df['VIX'], label='VIX (Implied)', color='coral', alpha=0.8)
axes[0].plot(df.index, df['RV_21'], label='21-Day Realised', color='steelblue', alpha=0.8)
axes[0].fill_between(df.index, df['RV_21'], df['VIX'], 
                      where=df['VIX'] > df['RV_21'], alpha=0.3, color='coral', label='Premium')
axes[0].legend()
axes[0].set_ylabel('Volatility (%)')
axes[0].set_title('VIX vs Realised Volatility')
axes[0].grid(alpha=0.3)

# Panel 2: Volatility Risk Premium
df['VRP'] = df['VIX'] - df['RV_21']
axes[1].plot(df.index, df['VRP'], color='purple', alpha=0.8)
axes[1].axhline(0, color='k', linestyle='--', linewidth=0.5)
axes[1].axhline(df['VRP'].mean(), color='red', linestyle='--', label=f'Mean: {df["VRP"].mean():.1f}%')
axes[1].fill_between(df.index, 0, df['VRP'], where=df['VRP'] > 0, alpha=0.3, color='coral')
axes[1].fill_between(df.index, 0, df['VRP'], where=df['VRP'] < 0, alpha=0.3, color='steelblue')
axes[1].legend()
axes[1].set_ylabel('VIX - Realised (%)')
axes[1].set_xlabel('Date')
axes[1].set_title('Volatility Risk Premium')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### Task 2.3: Quantify the Premium

In [ ]:
print("=== Volatility Risk Premium Statistics ===")
print(f"Mean Premium: {df['VRP'].mean():.2f}%")
print(f"Median Premium: {df['VRP'].median():.2f}%")
print(f"Std Dev: {df['VRP'].std():.2f}%")
print(f"% Days Premium > 0: {(df['VRP'] > 0).mean()*100:.1f}%")
print(f"Min Premium: {df['VRP'].min():.2f}%")
print(f"Max Premium: {df['VRP'].max():.2f}%")

> **Discussion Question 1**
>
> On what percentage of days is the premium positive? What does this
> tell us about selling volatility as a strategy?

## Part 3: Regime Analysis

### Task 3.1: Low vs High Volatility Regimes

Does the premium behave differently in calm vs stressed markets?

In [ ]:
# Define regimes based on VIX level
df['Regime'] = pd.cut(df['VIX'], 
                       bins=[0, 15, 25, 100], 
                       labels=['Low (<15)', 'Medium (15-25)', 'High (>25)'])

# Premium statistics by regime
regime_stats = df.groupby('Regime').agg({
    'VRP': ['mean', 'std', 'count'],
    'VIX': 'mean',
    'RV_21': 'mean'
}).round(2)

print("=== Premium by VIX Regime ===")
print(regime_stats)

### Task 3.2: Visualise Regime Differences

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Box plot by regime
df.boxplot(column='VRP', by='Regime', ax=ax)
ax.axhline(0, color='red', linestyle='--', linewidth=1)
ax.set_ylabel('Volatility Risk Premium (%)')
ax.set_xlabel('VIX Regime')
ax.set_title('Premium Distribution by Market Regime')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

> **Discussion Question 2**
>
> How does the premium vary across regimes? When is selling volatility
> most profitable? Most risky?

## Part 4: Forecasting Relationship

### Task 4.1: Does VIX Predict Realised Volatility?

In [ ]:
# Shift realised volatility forward to compare VIX today to RV next 21 days
df['RV_Forward'] = df['RV_21'].shift(-21)

# Scatter plot
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(df['VIX'], df['RV_Forward'], alpha=0.3, s=10)

# 45-degree line
lims = [0, 80]
ax.plot(lims, lims, 'r--', label='Perfect Forecast')

# Regression line
mask = ~(df['VIX'].isna() | df['RV_Forward'].isna())
slope, intercept, r, p, se = stats.linregress(df.loc[mask, 'VIX'], df.loc[mask, 'RV_Forward'])
x_line = np.linspace(10, 80, 100)
ax.plot(x_line, intercept + slope * x_line, 'b-', label=f'Fitted (R² = {r**2:.3f})')

ax.set_xlabel('VIX Today')
ax.set_ylabel('Realised Volatility (Next 21 Days)')
ax.set_title('VIX as a Predictor of Future Volatility')
ax.legend()
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== Forecasting Regression ===")
print(f"Slope: {slope:.3f}")
print(f"Intercept: {intercept:.3f}")
print(f"R-squared: {r**2:.3f}")

### Task 4.2: Forecast Errors

In [ ]:
df['Forecast_Error'] = df['VIX'] - df['RV_Forward']

print("=== Forecast Error Statistics ===")
print(f"Mean Error (Bias): {df['Forecast_Error'].mean():.2f}%")
print(f"MAE: {df['Forecast_Error'].abs().mean():.2f}%")
print(f"RMSE: {np.sqrt((df['Forecast_Error']**2).mean()):.2f}%")

> **Discussion Question 3**
>
> Is VIX a biased predictor of realised volatility? What is the
> direction of the bias, and why might this be?

## Part 5: Trading Implications

### Task 5.1: Simple Volatility Selling Strategy

**Strategy**: Sell volatility when VIX \> Realised Volatility by a
threshold.

In [ ]:
# Simple strategy: track cumulative "premium harvested"
# (This is illustrative, not a real trading strategy)

# Monthly rebalancing
df_monthly = df.resample('M').last()
df_monthly['Premium_Earned'] = df_monthly['VIX'].shift(1) - df_monthly['RV_21']

# Cumulative premium
df_monthly['Cumulative_Premium'] = df_monthly['Premium_Earned'].cumsum()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(df_monthly.index, df_monthly['Cumulative_Premium'], color='darkgreen', linewidth=2)
ax.axhline(0, color='k', linestyle='--', linewidth=0.5)
ax.set_ylabel('Cumulative Premium Harvested (%)')
ax.set_xlabel('Date')
ax.set_title('Hypothetical Cumulative Volatility Premium (Monthly)')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTotal Premium Harvested: {df_monthly['Cumulative_Premium'].iloc[-1]:.1f}%")
print(f"Average Monthly Premium: {df_monthly['Premium_Earned'].mean():.2f}%")

> **Risk Warning**
>
> This is a simplified illustration. Real volatility selling strategies
> involve significant tail risk — occasional large losses can wipe out
> accumulated gains. The 2020 COVID crash and 2008 crisis showed how
> quickly the premium can reverse.

## Deliverables

Complete and submit the following:

### 1. Data Summary Table

| Metric                 | Your Value |
|------------------------|------------|
| Average VIX            |            |
| Average 21-Day RV      |            |
| Average Premium        |            |
| % Days Premium \> 0    |            |
| R² (VIX vs Forward RV) |            |
| Mean Forecast Bias     |            |

### 2. Key Charts

Export and include: - VIX vs Realised Volatility time series - Premium
by regime boxplot - VIX vs Forward RV scatter plot

### 3. Written Analysis (200-300 words)

Address the following questions: 1. What is the average volatility risk
premium in your data? 2. How does the premium vary across market
regimes? 3. Is VIX a good predictor of future realised volatility? 4.
What risks would a volatility-selling strategy face?

### 4. Connection to GARCH

Compare your findings to the GARCH homework: - How does GARCH-estimated
volatility compare to VIX? - Which is more responsive to market
shocks? - When might you prefer one measure over the other?

## Optional extensions

If you want an advanced extension, you can also:

1.  **Term Structure**: Extract VIX futures (VX1, VX2, etc.) and analyse
    the volatility term structure. When is it in contango vs
    backwardation?

2.  **International Comparison**: Extract VSTOXX (European volatility
    index) and compare its premium to VIX.

3.  **Event Study**: Identify 3-5 major market events (COVID crash, Fed
    rate decisions) and analyse how VIX and realised volatility behaved
    around these events.

## References

-   Carr and Wu (2009) on variance risk premia
-   Bollerslev, Tauchen, and Zhou (2009) on expected stock returns and
    variance risk premia

Bollerslev, Tim, George Tauchen, and Hao Zhou. 2009. “Expected Stock
Returns and Variance Risk Premia.” *Review of Financial Studies* 22
(11): 4463–92. <https://doi.org/10.1093/rfs/hhp008>.

Carr, Peter, and Liuren Wu. 2009. “Variance Risk Premiums.” *Review of
Financial Studies* 22 (3): 1311–41.
<https://doi.org/10.1093/rfs/hhn038>.